In [14]:
!pip3 install  opencv-python --user

    100% |████████████████████████████████| 40.3MB 564kB/s ta 0:00:011    55% |█████████████████▊              | 22.4MB 1.2MB/s eta 0:00:15    63% |████████████████████▎           | 25.5MB 1.3MB/s eta 0:00:12
You are using pip version 19.0.3, however version 21.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Importing required modules

In [4]:
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.layers import Input, Dense
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.models import Model
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report,log_loss,confusion_matrix
from sklearn import tree, model_selection
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
import seaborn as sns
from sklearn.feature_selection import f_classif,SelectKBest
from sklearn.model_selection import KFold,cross_val_score,cross_validate
from matplotlib import pyplot
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import cv2

WEIGHT_LOC = './saved_model/'
IMG_LOC = './img/'

# Helper functions , used to computer the results in the report

In [2]:

## load training images 
def loadTrainingImages():
  tr_images = []
  tr_labels = []
  val_images = []
  val_labels = []


  #train_path = '/content/gdrive/MyDrive/Colab Notebooks/ML-Project/archive.zip (Unzipped Files)/chest_xray/train/'
  #val_path = '/content/gdrive/MyDrive/Colab Notebooks/ML-Project/archive.zip (Unzipped Files)/chest_xray/val/'

  for img in glob.glob(IMG_LOC +"normal/*.jpeg"):
      s = cv2.imread(img)
      #s = cv2.resize(s,(512,512))
      tr_images.append(s)
      tr_labels.append(0.0)
  for img in glob.glob(IMG_LOC+'p/*.jpeg'):
      s = cv2.imread(img)
      #s = cv2.resize(s,(512,512))
      tr_images.append(s)
      tr_labels.append(1.0)
  return tr_images,tr_labels

def plot_loss(history): 
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()
def plot_accuracy(history) :
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('Model Accuracy')
  plt.ylabel('acc')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()

import matplotlib.pyplot as plt
import seaborn as sns


#plot pca 
def plotPCA(x,nocomponents):
    fig= plt.figure(figsize=(7,7))
    pca = PCA(n_components=nocomponents)
    x_flatten = x.reshape(x.shape[0],x.shape[1]*x.shape[2]*3)

    pca.fit(x_flatten)
    cum_var_explained = []
    for i in range(len(pca.explained_variance_ratio_)):
        cum_var_explained.append(sum(pca.explained_variance_ratio_[:i]))

    plt.plot(list(range(0,len(cum_var_explained))),
                cum_var_explained, color="darkblue")
    idx_exp_95var = len(cum_var_explained) - 1
    idx_exp_90var = len(cum_var_explained) - 1
    idx = np.where(np.array(cum_var_explained) >= 0.95)
    print(idx)
    if idx[0].shape[0] > 0:
        idx_exp_95var = idx[0][0]
    idx = np.where(np.array(cum_var_explained) >= 0.99)
    print(idx)

    if idx[0].shape[0] > 0:
        idx_exp_99var = idx[0][0]

    #plt.plot([0,len(cum_var_explained)],[0.95,0.95],'--',color='darkblue')
    #plt.plot([idx_exp_95var,idx_exp_95var],[0,1],'--', color='darkblue')
    plt.plot(idx_exp_95var,0.95,'ro', color='red')
    plt.plot(idx_exp_99var,0.99,'ro', color='red')

    plt.xlabel('No. of components', fontsize=12)
    plt.ylabel('Percent Variance Explained', fontsize=12)
    plt.text(250, 0.92, '95%', fontsize=11,
          verticalalignment='center')
    plt.text(1250, 0.94, '99%',fontsize=11,
          verticalalignment='center')

    plt.plot()
    return pca 
def fitLogisticRegression(x,y):
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import classification_report, confusion_matrix
    x_flatten = x.reshape(x.shape[0],x.shape[1]*x.shape[2]*3)


    model = LogisticRegression(solver='liblinear', random_state=0).fit(x_flatten, y)
    return model
def fitANOVA(x,y):
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import classification_report, confusion_matrix
    x_flatten = np.array(x.reshape(x.shape[0],x.shape[1]*x.shape[2]*3))
    fs = SelectKBest(score_func=f_classif, k='all')
    fs.fit(x_flatten,y)
    idx_sorted      = np.argsort(fs.scores_)
    idx_sorted_desc = np.argsort(fs.scores_)[::-1]
    return idx_sorted_desc

def make_model_best(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    #x = data_augmentation(inputs)
    x = layers.Dense(32)(inputs)

    for size in [32,64]:
    # Entry block
      x = layers.Conv2D(size, kernel_size=3,strides=2,  padding="same")(x)
      x = layers.BatchNormalization()(x)
      x = layers.Activation("relu")(x)

    x = layers.Flatten()(x)
    
    outputs = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inputs, outputs)
def load_best_model():
    image_size =(128, 128,3)

    model = make_model_best(image_size,1)
    model = keras.models.load_model(WEIGHT_LOC)

    return model
def normalize(x):
    return np.array(x/255)


def create_ResNet50_model(input_tensor = Input(shape=(128, 128, 3)),
                          output_units = 1,optimizer='adam', weights='imagenet'):
  base_model = ResNet50(input_tensor=input_tensor, weights=weights)
  base_model.layers.pop()
  base_model.outputs = [base_model.layers[-1].output]
  #base_model.layers[-1].outbound_nodes = []
 # output = Dense(64, activation='relu')(base_model.output)
  predictions = Dense(output_units, activation='sigmoid')(base_model.output)
  model = Model(inputs=base_model.input, outputs=predictions)
  #for layer in base_model.layers:
    #layer.trainable = False
  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model



# Loading training data & fitting 2-covolutional neural network 

In [5]:
x,y = loadTrainingImages()

x = np.array(x)
y = np.array(y)
x = normalize(x)

model = make_model_best((128, 128,3),1)
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy']
)
history = model.fit(
    x = x,
    y = y,
    epochs=22#,
    
    #callbacks=[es_callback]
)
#print(x.shape)


Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
dense_6 (Dense)              (None, 128, 128, 32)      128       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
batch_normalization_6 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
activation_6 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 64)        256 

# Loading saved model 

In [6]:

model = load_best_model()
model.summary()


Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
dense_6 (Dense)              (None, 128, 128, 32)      128       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
batch_normalization_6 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
activation_6 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 64)        256 

2.3.1
